In [1]:
#Problema MBA
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

m = gp.Model('problema_MBA')

Set parameter Username


In [2]:
Produtos = ['Mesa', 'Cadeira', 'Estante', 'Armario', 'Cama']

In [3]:
Lucro = pd.Series([500, 200, 600, 900, 700], index = Produtos, name = 'Lucro')

Lucro.to_frame()

,Lucro
Mesa,500
Cadeira,200
Estante,600
Armario,900
Cama,700


In [4]:
restrições_de_capacidade = ['Materia prima', 'Horas', 'Armazenamento']

Capacidade = pd.Series([150, 100, 120], index = restrições_de_capacidade, name = 'restrições de capacidade')

Capacidade.to_frame()

,restrições de capacidade
Materia prima,150
Horas,100
Armazenamento,120


In [5]:
dicionario_capacidades = {'Mesa': [2, 1, 2],
                         'Cadeira': [1, 0.5, 1],
                         'Estante': [3, 2, 2],
                         'Armario': [3, 1.5, 2],
                         'Cama': [2, 1, 2]}

df_capacidades = pd.DataFrame(dicionario_capacidades, index = restrições_de_capacidade)

df_capacidades

,Mesa,Cadeira,Estante,Armario,Cama
Materia prima,2,1.0,3,3.0,2
Horas,1,0.5,2,1.5,1
Armazenamento,2,1.0,2,2.0,2


In [6]:
m = gp.Model('problema_MBA')
x = m.addVars(Produtos, vtype = GRB.INTEGER ,name = 'PRODUTOS')
m.update()
x

{'Mesa': <gurobi.Var PRODUTOS[Mesa]>,
 'Cadeira': <gurobi.Var PRODUTOS[Cadeira]>,
 'Estante': <gurobi.Var PRODUTOS[Estante]>,
 'Armario': <gurobi.Var PRODUTOS[Armario]>,
 'Cama': <gurobi.Var PRODUTOS[Cama]>}

In [7]:
#Restrições de produção
restricao_producao_mesa = m.addConstr(x['Mesa']>=10, name = 'restrição de produção de mesas')
restricao_producao_cadeira = m.addConstr(x['Cadeira']>=20, name = 'restrição de produção de cadeiras')
restricao_producao_estante = m.addConstr(x['Estante']>=5, name = 'restrição de produção de estantes')
restricao_producao_cama = m.addConstr(x['Cama']>=15, name = 'restrição de produção de camas')
m.update()

In [8]:
#Restrições de capacidade
rc_matprima = m.addConstr(2*x['Mesa']+1*x['Cadeira']+3*x['Estante']+3*x['Armario']+2*x['Cama']<=150, name = 'restriçao de matéria prima') 
rc_horas = m.addConstr(1*x['Mesa']+0.5*x['Cadeira']+2*x['Estante']+1.5*x['Armario']+1*x['Cama']<=100, name = 'restrição para as horas') 
rc_armaz = m.addConstr(2*x['Mesa']+1*x['Cadeira']+2*x['Estante']+2*x['Armario']+2*x['Cama']<=120, name = 'restrição para armazenamento')
m.update()

In [9]:
#Criando a função objetivo
m.setObjective(500*x['Mesa']+200*x['Cadeira']+600*x['Estante']+900*x['Armario']+700*x['Cama'], GRB.MAXIMIZE)

In [10]:
#Escrevendo o problema, pra checar se está correto
m.write('problema MBA.lp')

In [11]:
#Otimizando
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 19 nonzeros
Model fingerprint: 0xfffa1c09
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [2e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+02]
Presolve removed 7 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 40500 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.050000000000e+04, best bound 4.050000000000e+04, gap 0.0000%


In [12]:
#Obtendo os valores
xvals = {k: v.x for k,v in x.items() if v.x > 0}
xvals

{'Mesa': 10.0, 'Cadeira': 20.0, 'Estante': 5.0, 'Armario': 20.0, 'Cama': 15.0}

In [13]:
#Mostrando o resultado da função objetivo
obj = m.getObjective()
print(f"O valor da função objetivo é {round(obj.getValue(),2)}")

O valor da função objetivo é 40500.0
